In [ ]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import models

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 64
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=False, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=False, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [ ]:
model = models.LinearClassifier()

images, labels = next(iter(testloader))

probs = torch.exp(model(images)) 
# check output from model has the right size: batch_size x #classes
print(probs.shape)

In [ ]:
t1 = torch.tensor([3, 15, 1])
t2 = torch.tensor([[1], [3], [1]]) 
r = t1 == t2
print(r.shape)
r = t1 == t2.view(*t1.shape)
print(r.shape)
print(r)

In [ ]:
pred_probs, pred_labels = probs.topk(1, dim=1)
print(f'size of predicted labels: {pred_labels.shape}')
print(f'size of training labels:  {labels.shape}')

# print first 3 labels, model is not trained yet!
print(pred_labels[:3])
print(labels[:3])

#comparing tensors of same size
train_labels = labels.view(*pred_labels.shape)
equals = pred_labels == train_labels
print(f'result should have size as labels:[64,1]: {equals.shape}')
print(equals[:3])


In [ ]:
print(f'length trainloader: {len(trainloader)}, *batch_size: {batch_size*len(trainloader)}')
print(f'length testloader: {len(testloader)}, *batch_size: {batch_size*len(testloader)}')

In [ ]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

In [ ]:
model = models.LinearClassifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# train the model
epochs=5

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    else:
        test_loss = 0
        accuracy = 0

        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
